In [1]:
import numpy as np
import pydub
import requests
import os
import json
import noisereduce as nr
import librosa
import librosa.display
from scipy.io import wavfile
import matplotlib.pyplot as plt

from PreprocessingFunctions import mp3_to_np, np_to_mp3, find_max_window, extract_best_windows, convert_mp3s_to_wav, get_spectro_from_wav

#### Download select mp3 files for local machine testing

In [4]:
PATH = "./data/costa_rica/mp3/"

for filename in os.listdir("./data/costa_rica"):
    if filename.endswith(".json"):
        with open("./data/costa_rica/" + filename, 'r') as speciesFile:
            species_data = json.load(speciesFile)
            for i in range(0, len(species_data)):
                d = species_data[i]
                if d["file-name"].endswith(".mp3") and d["smp"]=="48000" and d["q"] != "E":
                    mp3_url = d["file"]
                    r = requests.get(mp3_url, allow_redirects=True)
                    open(PATH + filename[:-5] + f"_{i}.mp3", 'wb').write(r.content)

#### Make data windows for Costa-Rica data

In [ ]:
PATH_TO_COSTA_MP3 = "./data/costa_rica/mp3"

for filename in os.listdir(PATH_TO_COSTA_MP3):
    if filename.endswith(".mp3"):
        sr, numpy_audio_array = mp3_to_np(PATH_TO_COSTA_MP3 + "/" + filename)
        if len(numpy_audio_array.shape) == 2:
            numpy_audio_array = np.mean(numpy_audio_array, axis=1)
        windows = extract_best_windows(numpy_audio_array, sr)
        for i in range(0, len(windows)):
            np_to_mp3("./data/costa_rica/mp3_windows/" + filename[:-4] + f"_{i}.mp3", sr, windows[i])

#### Sort Costa data into training and validation splits. Testing data will come from real recordings

In [7]:
import random
import shutil

SPEC_PATH = "data/costa_rica/spectrogram"
TRAIN_PATH = "data/costa_rica/train"
VAL_PATH = "data/costa_rica/validation"
TRAIN_RATIO = .9

if not os.path.exists(TRAIN_PATH):
    os.mkdir(TRAIN_PATH)
if not os.path.exists(VAL_PATH):
    os.mkdir(VAL_PATH)
    
for directory in os.listdir(SPEC_PATH):
    if not os.path.exists(TRAIN_PATH+"/"+directory):
        os.mkdir(TRAIN_PATH+"/"+directory)
    if not os.path.exists(VAL_PATH+"/"+directory):
        os.mkdir(VAL_PATH+"/"+directory)
    file_list = os.listdir(SPEC_PATH+"/"+directory)
    random.shuffle(file_list)
    t_idx = int(len(file_list)*TRAIN_RATIO)
    train_files = file_list[:t_idx]
    val_files = file_list[t_idx:]
    train_dest = TRAIN_PATH+"/"+directory
    val_dest = VAL_PATH+"/"+directory
    for filename in train_files:
        if not filename.endswith(".ipynb_checkpoints"):
            shutil.copy(SPEC_PATH+"/"+directory+"/"+filename, train_dest)
    for filename in val_files:
        if not filename.endswith(".ipynb_checkpoints"):
            shutil.copy(SPEC_PATH+"/"+directory+"/"+filename, val_dest)
            

In [4]:
# load data
data, sr = librosa.load("data/costa_rica/wav/Broad-billed_Motmot_105_4.wav", sr=None)
# perform noise reduction
reduced_noise = nr.reduce_noise(y=data, sr=sr)
wavfile.write("data/costa_rica/denoised_wav/Broad-billed_Motmot_105_4_denoise.wav", sr, reduced_noise)

In [6]:
get_spectro_from_wav(audio_clips=["Broad-billed_Motmot_105_4_denoise.wav"], wav_path="data/costa_rica/denoised_wav/", export_path="data/costa_rica/")

0.0
